# Summary
This interactive Jupyter Notebook has the objective of evaluating the different relationships and basic processes related to the interception and absorption of irradiance.

We will make use of simulations models, particularly radiative transfer models at both leaf and canopy levels.

# Instructions
Read carefully all the text and follow its instructions.

Once each section is read, run the jupyter code cell underneach (marked as `In []`) by cliking the icon `Run`, or pressing the keys ALT+INTRO of your keyboard. A graphical interface will then be displayed which you could interact with and performe the assigned tasks.

As a starters, please run the following cell in order to import all the required libraries in this notebook. Once the cell below is run an aknowledgement message should be printed on screen.

In [ ]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed
from IPython.display import display
from functions import radiation_and_available_energy as fn
import numpy as np

# The energy balance
The evapotranspiration is basically an exchange of heat and water between the land surface and the atmosphere. Therefore, one of the key factors for its retrieval is the estimation of the available energy at the land surface. 

![Earth energy balance](./input/figures/radiation_balance.png "The Earth energy balance").

Depending on the surface water status, this available energy is utilized for evaporating water (ET or $\lambda E$ in terms of energy fluxes) and/or for increasing its temperature ($H$).

$$\lambda E + H = R_n - G$$

where $G$ is the heat trasmitted and stored into the ground, and $R_n$ is the net radiation.

The net radiation ($R_n$) can be computed as:

\begin{equation}
R_n = S^\downarrow \left(1 - \alpha\right) +  \epsilon \left(L^\downarrow - \sigma LST^4\right)
\end{equation}

where $S^\downarrow$ and $L^\downarrow$ are respectively the shortwave and longwave incoming irradiances, which can be obtained from meterological stations, numerical weather models or meteorological satellites. $\alpha$ and $\epsilon$ are surface albedo and emissivity, $LST$ es the Land Surface Temperature, y $\sigma\approx5.67E-8$ (W/m²K⁴) is the Stefan-Boltzman constant. $\alpha$, $\epsilon$ and $LST$ can as well be estimated, more or less accurately, from Earth Observation.

Considering the larger magnitude of shortwave radiance ($S$) compared to the longwave radiance ($L$), we will put more emphasis in this notebook on the former one. In particular on the canopy and leaf properties that influence albedo and radiation partitioning between soil and canopy.

On the other hand, $G$ is usually of much lower magnitude, being usually negligible at daily or even longer scales. When working at shorter timescale, e.g. when estimating $\lambda E$ at the satellite overpass, $G$ is usually estimated as a fraction of net radiation (e.g. $G \approx 0.1 R_n$ for dense canopies), but also as a fraction of net radiation at the soil surface ($G \approx 0.1 R_{ns}$).

# Beer-Lambert law
The Beer-Lambert law is a physical law that allows describing and interpreting most of the radiative processes of our interest: When an electromagnetic beam passes through a medium containing an absorber of radiation, this light is attenuated proportionally to the concentration of such absorber ($\kappa$) and to the optical path length of the beam $\ell$):

\begin{equation}
\tau = e^{-\kappa\ell}
\end{equation}

In the following interactive graph you could see the effect of this law. Youcould modify the values of the extinction cofficient, which increases with the concentration of the abosorbing material in the medium. 

The plot will display the fraction of light that is transmitted through the medium as the beam path becomes longer:

In [ ]:
w_lambert = interactive(fn.beer_lambert_law, kappa=fn.w_kappa, length=fixed(np.linspace(0, 10, 50)))
display(w_lambert)

This law permits for instance explain the attenuation of solar radiation from the estratosphere to the Earth's surface: the air is composed by gases and particles that absorb radiation (most of them at certain wavelengths such as the Ozone or the Oxygen). The higher the Sun is in the sky, the path length across the atmopshere will be shorter, and thus the solar irradiance reaching the land is greater.

![Sun optical path lenght](./input/figures/airmass.png "Sun optical path lenght").

## Application to the fraction of intercepted radiation
Beer-Lambert law also allows the analysis or estimation of the radiation that is intercepted by the foliage in a canopy, as well as the radiation transmitted towards the ground.

Assuming that a canopy is primarily composed by leaves, and that these leaves intercept and absorbed radiation (mostly photosynthetically active radition, PAR), we could evaluate how much radiation is intercepted by the canopy according to the foliage density. The latter is usually defined as the Leaf Area index: half of the total leaf surface per unit ground. The larger the LAI, more likelly is that sun beams were intercepted by one or more leaves, and thus less likely is that solar beams reached the ground. 

Likely, the solar elevation angle ($\beta$ in the previous graph), or its complementary the solar zenith angle (SZA), plays a relevant role. The lower the sun is from the horizon, the longer the path of the sun beams will be  when passing through the canopy, and thus the larger the light attenuation.

Since leaves are solid elements with a finite size (as oppposed to the gases in the atmosphere), their orientation with respect to the nadir also plays a relevant role. Therefore the Beer-Lambert law must be adapted to account for this effect. In the next graph you could experience with this phenomenon. You could change both the amount of leaves in terms of LAI and the dominant zenith orientation of leaves, from canopies with leaves predominantly vertical (LIDF$\approx$90º) to leaves predominantly horizontal (LIDF$\approx$0º).

In [ ]:
w_fipar = interactive(fn.plot_fipar, lai=fn.w_lai, leaf_angle=fn.w_leaf_angle)
display(w_fipar)

Whatch how the largest fraction of intercepted radiation (or the largest light attenuation) always happen at higher solar zenith angles (when the path lenght of the solar beams is larger), but that the attenuation rated depends mostly on the amount of leaves (higher LAI) and its dominant orientation.


LAI depends mainly on the plant development stage and on the plant growth, while the leaf orientation is more specific for the different species or plant functional types. 

As a rule of thumb we could expect that plants with leave predominantly vertical (i.e. erectophylle) are usually adapted to climates with high irradiance levels, with the aim of avoiding large exposure and interception around noon, when irradiance is maximum. On the other hand, plants with leaves predominantly horizontal tend to thrive in more shaded conditions or with low irradiance levels, in order to maximize the interception of solar radiation along the daytime. Between these two extremes, most of the plants show an angular distribution more or less uniform, for those cases the calculation of intercepted radiation can simplify as:

\begin{equation}
fIPAR = 1 - \exp \left(\frac{-0.5 LAI}{\cos\theta_s}\right)
\end{equation}
siendo $\theta_s$ el ángulo zenital solar.

> If you want to know more details on these calculations you can check the source code [here](https://github.com/hectornieto/pyTSEB/blob/6dd5dffff08bc1f08edb3e89b4a79879e348146b/pyTSEB/TSEB.py#L1587 "https://github.com/hectornieto/pyTSEB/blob/6dd5dffff08bc1f08edb3e89b4a79879e348146b/pyTSEB/TSEB.py#L1587").

# The albedo
As we have previously mentioned, the albedo ($\alpha$) is the key variable that determines the amount of interepcted light that is absorbed by the surface. $\alpha$ is defined as the proportion of incident shortwave radiation that is reflected by the surface. The shortwave net radiation ($S_n$) is therfore the balance between the incident shortwave irradiance ($S^\downarrow$) and the reflected shortwave radiance ($S^\uparrow = \alpha S^\downarrow$)

The spectral properties of the surface are key in determining the albedo, for instance the fresh snow shows large values ($\alpha\approx 0.9$) and thus reflect most of the solar irradiance, while oceans have much lower values ($\alpha\approx0.05$), absorbing a large amount of solar irradiance.

The leaves, due to their photosynthetic activity absorb a large proportion of light due to the presence of cholorphylles, as well as other leaf pigments. For that reason leaves yield relative low albedo values ($\approx0.15$). On the other hand, soils can have a large range of albedo values, depending on its mineral composition, texture and topsoil moisture. Therefore, the albedo of a vegetated surface will depend not only on leaf chorophyll concentration but also on canopy density and, in a lesser degree on soil albedo in situation of sparse vegetation or initial growth stages.

> You will probably work more on this topic in other sessions of this training course.

## Vegetation anisotropy
Most of the Earth surface show certain anisotropic behaviour when reflecting radiation. I.e. they scatter more or less radiation depending on scattering direction. The most extreme cases would be the specular surfaces (such as mirrors or still water bodies or ice), which scatter most the irradiance in the opposite direction as the incidence angle. The opposite case would be the lambertian surfaces, which apparent brighness/scattering is the same regardless the direction.

![Lambertian and specular scattering](./input/figures/lambertian_and_specular_reflection.png "Lambertian and specular scattering").

> Unpolished wood would be closer to a lambertian surface, while polished and barnished wood would behave more as a specular/non-lambertian surface. 

Vegetation, as it is mainly composed by an array of leaves, is also affected by this anisotropic behavioiur. Therefore, plants will reflect radiation differently depending on the illumination geometry (i.e. the solar  position) and the scattering direction (i.e. the sensor position), possibly changing itd albedo.

You will experience this effect in the following plot. It depicts two polar graph, one for the photosyntheticall active radiation (PAR, 400-700$\mu$m) and a second one for the shortwave radiation (SW, 400-2500$\mu$m). These plots represents the the reflectance factor simulated for a homogeneous canopy, using the radiative transfer models PROSPECT-D and 4SAIL.

The concentric rings represent the range of zenith angles (zenith=0º at the plot origin, and 90º at the outermost ring). The radii of the plots show the azimuth angles (0º bearing North, 90º bearing East, 180º bearing South, and 270º bearing West). The solar position is depicted in the graph with a star. By default the sun is placed at solar noon in Northern latitudes (180º azimuth), at solar zenith angle of 37º. The plots also print  above the integrated value of the reflectance factors, which is actually the albedo.

> If you want to know more details on these calculations you can check the source code [here](https://github.com/hectornieto/pyPro4SAIL "https://github.com/hectornieto/pyPro4SAIL").

In [ ]:
w_bidirectional = interactive(fn.bidirectional_reflectance,
                              cab=fn.w_cab, cw=fn.w_cw, lai=fn.w_lai, leaf_angle=fn.w_leaf_angle, 
                              sza=fn.w_sza, saa=fn.w_saa, skyl=fn.w_skyl, soil_type=fn.w_soil)
display(w_bidirectional)                             

* Modify the chlorophyll content - `Cab`. ¿How do reflectance factors and albedo change in the PAR region (400-700$\mu$m) and in the solar spectrum (400-2500$\mu$m)?
* Likewise, modify the leaf water content- `Cw`. ¿How do reflectance factors and albedo change in the PAR region (400-700$\mu$m) and in the solar spectrum (400-2500$\mu$m)?
* Modify the solar position - `SZA` and `SAA`. On one side you will see a peak on reflectance around the solar position (the star in the plot). This is the so-called hotspot, or the area most illuminated in the canopy. On the other side, the lowest values are in the opposite site, since leaves in this area are more occluded by the rest of the canopy. Besides the albedo values also vary with the solar poisition, i.e. albedo changes along the day.
* This variation of albedo with solar  positoin is stronger with lower values of diffuse radiation. Increase the ration of diffuse radiation `skyl`, which would simulate cloudy days. With totally overcast days (`skyl=1`) you will notice that the solar position is no longer relevant for the albedo or reflectance factors.

# Net shortwave radiation
Previously we have seen that LAI, leaf angle distribution and chlorophyll concentration are the probably the most relevant variables that determine the absortion of shortwave irradiance in a crop.

In the next plot we could see the daily trend of net shortwave radiation for a herbaceous crop horizontally homogeneous (such a wheat field). These simulations assume that the cloudiness remains constant along the day and thus shortwave irradiance displays a sinusoidal curve: Should cloudiness changed along the day, the shortwave irradiance would display a series of valleys and peaks, according to whether the sun is occluded by clouds or not respectively.

> On the other hand, these simulations assume a constant ratio of diffuse irradiance along the day (fixed by the value set by `Skyl`). This is unrealistic in most cases, since arund sunrise and sundown diffuse radiation tends to be larger even under clear skies. But for simplicity in the simulations we made this assumption.

The plot depicts the net radiation at the land surface ($S_n$, black line) as well as the net radiation partitioning between crop ($S_{n,C}$, green line) and ground ($S_{n,S}$, yellow line). The secondary axis displays the albedo in blue line.

> $S_{n,C}+S_{n,S}=S_n$
>
> $\alpha = 1 - S_n/S\downarrow$

In [1]:
w_sn = interactive(fn.plot_net_solar_radiation,
                   lai=fn.w_lai, leaf_angle=fn.w_leaf_angle, h_c=fixed(1), f_c=fixed(1),
                   sdn_day=fn.w_sdn,
                   row_distance=fixed(1), row_direction=fixed(1), skyl=fn.w_skyl, 
                   fvis=fixed(0.55), lat=fn.w_lat, cab=fn.w_cab, cw=fn.w_cw, soil_type=fn.w_soil)
display(w_sn)

NameError: name 'interactive' is not defined

> All parameters in all the plots are synchronized: any parameter change in one of the interactive plots is as well changed in the other plots, being all the graphs updated accordingly. This will allow a quick intercomparison between graphs and simulations.

* While keeping the chlorophyll and water content constant observed how net radiation barely changes with variations of LAI. However, the partitioning of net radiation between the canopy and soil do drastically change with LAI. This will have implications since LAI is key for the radiation partitioning and thus in the evaporative/transpirative capacity of soil and vegetation as well as on the photosynthetis.

* Keep now relatively low values of LAI (<1.5) and watch now the effect of leaf angular distribution. More vertical leaves (`LIDF`$\rightarrow 90^{\circ}$) would provoke a reduction of intercepted, and thus absorbed, radiation near solar noon. Watch now that if diffuse radiation increases (`Skyl`$\rightarrow$1) this effect is no longer evident, in this case most of the radiation is diffuse and hence comming equally from any direction of the hemisphere, making leaf inclination less relevant.

* Change the soil type and see how soil albedo affects net radiation, in particular at lower values of LAI when soil is more exposed to solar irradiance.

> If you want to know more details on these calculations you can check the source code [here](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/net_radiation.py#L543 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/net_radiation.py#L543").

## Shortwave radiation in row crops
So far all simulations and processed we have evaluated assumed a canopy or crop horizontally homogeneous, or at least dense enough to approach to a homogeneous crop. However, what happens when one is working with row crops or isolated canopies, such as in vineyards or orchards?

The hard fact is that everything is much more complex as from one side a canopy is usually clumped around a stem and branches, leaving some parts of the ground fully exposed. On the other side, in row crops, the dimension of the canopy/trellis system and the orientation of the rows also affect on how light is intercepted and transmitted between the crop and the ground.

In order to evaluate this situation, it is needed to consider additional crop strucutural parameters:

* The canopy height ($h_c$), since higher canopies will cast longer shadows in the interrow.
* The fraction of canopy occupied by the ground ($f_c$), or the ratio between the canopy/trellis width and the row spacing (L).
* The azimuth row orientation, with respect to the North. 0º will indicate that the rows are oriented S-N, negative values indicate that the rows go from NW to SE, and positive values from NE to SW, with -90º and/or 90º showing rows oriented E-W.

In the following interactive plot we have added therefore these parameters to see their effect on net radiation. In addition, to ease the comparison with previous plot, we have included a second graph that shows the fraction of absorbed radiation for both a horizontally homogeneous crop and a row crop.

In [ ]:
w_sn = interactive(fn.plot_net_solar_radiation,
                   lai=fn.w_lai, leaf_angle=fn.w_leaf_angle, h_c=fn.w_hc, f_c=fn.w_fc,
                   row_distance=fn.w_interrow, row_direction=fn.w_psi, sdn_day=fn.w_sdn, skyl=fn.w_skyl, 
                   fvis=fixed(0.55), lat=fn.w_lat, cab=fn.w_cab, cw=fn.w_cw, soil_type=fn.w_soil)
display(w_sn)

> All parameters in all the plots are synchronized: any parameter change in one of the interactive plots is as well changed in the other plots, being all the graphs updated accordingly. This will allow a quick intercomparison between graphs and simulations.

* The sun rises from the eastern side, at noon reaches its zenith towards the North (or South depending on whether we are North of South of the Equator), and sunset is from the western side. Taking into account this fact, observe the effect of changing the row orientation from E-W to S-N. There should be a drop in canopy net radiation when the sun is parallel to the rows, as in this case the radiation interception by the canopy is lower.

* Observe that with larger canopy fractions ($f_c$, or wider canopies related to the row spacing) the crop behaves closer to the homogeneus crop from the previous simulation.

* There is an interaction effect between canopy height and row separation (`L`). Narrower rows in lower crops display a similar effect that taller crops in wider rows.

* Finally, as we have seen in previous interactive plots, these effect are minimized at larger proportion of diffuse radiation.

> If you want to know more details on these calculations you can check the source code [here](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/clumping_index.py#L124 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/clumping_index.py#L124").

# Net longwave radiation
Net longwave radiation ($L_n$) corresponds to the thermal emission of radiation from the land surface and the asborption of thermal irradiance from the atmosphere. $L_n$ depends thus on the surface's temperature and its emissivity as well as on the atmospheric thermal emission ($L\downarrow$), which at the same time depends on the atmospheric temperature and emissivity.

> $L_n = \left(1 - \epsilon_{surf}\right) L\downarrow - \epsilon_{surf} \sigma T_{surf}^4$

In the next interactive plot, we will see how net longwave radiation changes as function of atmospheric conditions (air temperature and humidity) and surface's temperature and emissivity. Since surface temperature also depends on given atmospheric conditions, we simulated various ranges of surface temperatures in the x-axis, from fully watered crops in which its surface temperature is closer to the air temperature, to stressed crops and bare/sparse areas in which surface temperature is significantly hotter than the air.


In [ ]:
w_ln = interactive(fn.plot_longwave_radiation, t_air=fn.w_tair, hr=fn.w_hr, delta_t=fixed(np.linspace(-1, 20, 50)),
                   emiss=fn.w_emiss)
display(w_ln)

* You could observe that net longwave radiation yield values at a lower magnitude than the net shortwave radiation ( speciallyl for for clear skies around solar noon time). Only in cases with high temperatures and humidity net radiation exceeds 400 W/m².

* Finally the surface emissivity, which values usually range from 0.95 in arid areas with scarce vegetation to 0.99 in dense vegetation surfaces, has a lesser effect in the calculation of net longwave radiation. Therefore its estimation for computing net radiation does not require for a great accuracy.

# Conclusions
In this exercise we have seen that net shortwave radiation depends mainly on:
1. Shortwave irradiance, which can be obtained from meteorological stations or from numerical weather models (e.g. [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview "https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview")).
2. Surface albedo

Furthermore, the surface albedo mainly depends on:
1. Leaf and soil spectral properties, mainly leaf chlorophyll as main absorber pigment in the PAR region.
2. Canopy structural characteristics, mainly LAI but also leaf angle distribution.
3. In addition, for row crops, the row architecture may result crucial in characterizing the radiation partitioning between ground and canopy.

The net longwave radiation has a lower contributino to the globar net radiation ($R_n$), and its estimation is usually simpler, requiring:
1. Incident longwave irradiance, which can be obtained from meteorlogical stations or from numerical weather models (e.g. [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview "https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview")).
2. Surface emssivity and temperature, which both can be obtained from thermal infrared Earth Observation data.

These analyses will allow you to better understand the radiative part of the energy balance. In addition, these simulations would permit evaluating the cost/benefit of using simpler or more sophisticated models for estimating net radiation and its partitioning between soil and canopy in strucutrally complex canopies.

In the next exercise you will work on the other relevant factor of the energy balance that can be estimated with themal infrared Earth Observation, the [sensible heat flux](./EN_turbulence_and_sensible_heat_flux.ipynb).